In [1]:
from PIL import Image
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from time import time
%matplotlib inline

In [2]:
categorias=[]
labels= []
imagenes = []
categorias=os.listdir('../data/archive/raw-img/')
print(categorias)
x=0
tic = time()
for directorio in categorias:
    for imagen in os.listdir('../data/archive/raw-img/'+directorio):
        img= Image.open('../data/archive/raw-img/' + directorio + '/' + imagen).resize((100, 100))
        #print(np.array(img).shape)
        if len(np.array(img).shape) == 3:
            img = np.mean(np.array(img), axis=2)
        else:
            img = np.array(img)
        #print(img.shape)
        #print(directorio,imagen)
        imagenes.append(img)
        labels.append(x)
        #break
    #break
    x += 1
    
imagenes_matrix = np.asarray(imagenes)
imagenes_matrix.shape
toc = time()
print("tiempo de computo:", toc-tic, "[s]")
print("etiquetas", np.unique(np.array(labels)))

['gallina', 'gatto', 'mucca', 'elefante', 'scoiattolo', 'farfalla', 'ragno', 'pecora', 'cavallo', 'cane']
tiempo de computo: 50.65620183944702 [s]
etiquetas [0 1 2 3 4 5 6 7 8 9]


In [3]:
np.unique(np.array(labels))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
y_train = tf.one_hot(labels, depth=10)
y_train.shape

TensorShape([26179, 10])

In [5]:
y_train.numpy()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [6]:
#mezclo los datos
X_train_tf, y_train_tf = shuffle(imagenes_matrix, y_train.numpy())
X_train_tf.shape, y_train_tf.shape

((26179, 100, 100), (26179, 10))

(batch, 100, 100, 1)

In [7]:
X_train_cv = np.expand_dims(X_train_tf, axis=3)
X_train_cv.shape

(26179, 100, 100, 1)

- Escalamos las imagenes

In [8]:
minimum = np.min(X_train_cv)
maximum = np.max(X_train_cv)
X_train_cv = (X_train_cv-minimum)/(maximum-minimum)

#### Red base

In [13]:
model= tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(100,100)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')

    ])

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train_cv, y_train_tf, batch_size = 128 , epochs=30, verbose=0)

#### Modelo 2

In [12]:
model = tf.keras.models.Sequential()#probar con border_mode='valid', same , 8 y 6
model.add(tf.keras.layers.Convolution2D(64, (3, 3), padding='same', input_shape=(100, 100, 1), activation='relu'))
model.add(tf.keras.layers.Convolution2D(16, (5, 5), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
model.add(tf.keras.layers.Convolution2D(16, (5, 5), padding='same', activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(.01)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=10, activation="softmax", kernel_regularizer = tf.keras.regularizers.l2(.01)))

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 64)      640       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 100, 100, 16)      25616     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 50, 50, 16)        6416      
_________________________________________________________________
flatten_2 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               5120128   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [ ]:
model.fit(X_train_cv, y_train_tf, batch_size=512 , epochs=3)